# Feature extraction CODE FOR PAPER

In [2]:
#import necessary libraries
import os
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50

# Define the paths
dataset_path = "C:/Users/cart.lab2/dataset1" #Root directory of the dataset
output_path = "C:/Users/cart.lab2/Frames" #Directory to save the extracted frames
features_path = "C:/Users/cart.lab2/features.pt" #Path to save the extracted features as tensors
labels_path = "C:/Users/cart.lab2/labels.txt" #Path to save the labels as a text file

# Create the ResNet50 model
model = resnet50(pretrained=True) # Load the pre-trained ResNet-50 model
model = model.eval() # Set the model to evaluation mode (no gradient computation)

# Define a sequence of transformations to be applied to each frame
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Initialize lists to store features and labels
features = [] #List to store the extracted features from frames
labels = [] #List o store the corresponding labels(Anomaly or Normal)

# Iterate over the folders
for folder_name in ["Anomaly", "Normal"]:
    folder_path = os.path.join(dataset_path, folder_name)
    if not os.path.exists(folder_path):
        continue

    # Iterate over the videos in the folder
    video_files = os.listdir(folder_path)
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)

        # Open the video file
        video_capture = cv2.VideoCapture(video_path)
        frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT)) # Get the total number of frames in the video

        # Extract frames and features
        for frame_index in range(frame_count):
            success, frame = video_capture.read() # Read the next frame from the video
            if not success:
                break

            # Extract features using ResNet50
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Convert frame from BGR to RGB (as ResNet-50 expects RGB)
            frame_tensor = transform(frame) # Apply the defined transformations to the frame
            frame_tensor = torch.unsqueeze(frame_tensor, 0) # Add a batch dimension (required by ResNet-50)
            with torch.no_grad():
                feature_tensor = model(frame_tensor)  # Pass the frame through the ResNet-50 model

            # Append features and labels
            # Convert the feature tensor to a numpy array and append
            features.append(feature_tensor.squeeze().numpy())
            
            # Append the corresponding label (Anomaly or Normal)
            labels.append(folder_name)

            # Save extracted frame
            frame_output_path = os.path.join(output_path, f"{folder_name}_{video_file}_{frame_index}.jpg")
            cv2.imwrite(frame_output_path, frame)

        # Release the video capture object
        video_capture.release()

# Convert features to tensors
features_tensor = torch.tensor(features)

# Save features as tensors
torch.save(features_tensor, features_path)

# Save labels in a text file
with open(labels_path, 'w') as f:
    for label in labels:
        f.write(label + '\n') 

C:\Users\cart.lab2\AppData\Local\Temp\ipykernel_8312\2381300825.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\utils\tensor_new.cpp:204.)
  features_tensor = torch.tensor(features)


In [4]:
features

tensor([[-4.8009, -1.2275, -3.4029,  ..., -1.9273, -1.0958,  1.6958],
        [-4.8076, -1.2441, -3.3980,  ..., -1.9423, -1.0766,  1.7222],
        [-4.8022, -1.2364, -3.3939,  ..., -1.9411, -1.0752,  1.7155],
        ...,
        [-1.8786,  1.6181,  0.0378,  ..., -1.1894,  0.6525, -1.3284],
        [-1.8312,  1.4049,  0.1747,  ..., -0.9394,  0.4374, -1.4905],
        [-1.8601,  1.3853,  0.1716,  ..., -0.9265,  0.4233, -1.4928]])